In [66]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from string import punctuation
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [67]:
driver = webdriver.Chrome('C:/Users/bandi/Desktop/Text Analytics/TA Session/chromedriver_win32/chromedriver')
driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p702')

In [3]:
#Creating empty data frame to store user_id, dates and comments from ~5K users.
comments = pd.DataFrame(columns = ['Date','user_id','comments'])

j = 702
while (j>=1):
    # Running while loop only till we get 5K comments 
    if (len(comments)<5000):
        url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p' + str(j)
        driver.get(url)
        ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
        comment_ids = []
        for i in ids:
            comment_ids.append(i.get_attribute('id'))

        for x in comment_ids:
            #Extract dates from for each user on a page
            user_date = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
            date = user_date.get_attribute('title')

            #Extract user ids from each user on a page
            user_title = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[1]/span[1]/a[1]')[0]
            userid = user_title.get_attribute('title') 

            #Extract Message for each user on a page
            user_message = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[3]')[0]
            comment = user_message.text
           #Adding date, userid and comment for each user in a dataframe    
            comments.loc[len(comments)] = [date,userid,comment]
        j=j-1
    else:
        break
        

In [ ]:
import copy
comments_copy = copy.deepcopy(comments)

In [30]:
def remove_space(s):
    return s.replace("\n"," ")
comments_copy['comments'] = comments_copy['comments'].apply(remove_space)

In [31]:
comments_copy.to_csv('comments.csv', header=True, sep=',') 

In [40]:
def removepunc(item):
    for p in punctuation:
        item = item.replace(p,'')
    return item

def remove_numbers(s):
    return re.sub("\S*\d\S*", "", s).strip()

def lowerize(x):
    return x.lower()

comments_copy['comments_clean'] = comments_copy['comments'].apply(removepunc).apply(lowerize)

In [41]:
comments_copy['comments_clean'][2]

'circlew saidthe lease rate is the factor that stops me cold from taking the leap since i dont track my cars it would do fine for my needs but i would assume an m2 would surpass the stinger on the track compared to the giuliayes the completely noncomparable m2 would beat the stinger on track as would a corvette or sportbike btw17 f150 crew 27 67 coronet rt 14 towncountry limited 09 lr2 hse 44car history and counting'

In [89]:
models = pd.read_csv("models.csv", header = None, names = ['brand','model'], encoding='windows-1252')

models['brand'] = models['brand'].apply(removepunc)

def model_to_brand(s):
    for i in models.index.values:
        s = s.replace(models["model"][i],models["brand"][i])
    return s
comments_copy['comments_model_replace'] = comments_copy['comments_clean'].apply(model_to_brand)

In [90]:
comments_copy['comments_model_replace'][2]

'circlew saidthe lease rate is the factor that stops me cold from taking the leap since i dont track my car it would do fine for my needs but i would assume an m2 would surpass the stinger on the track compared to the giuliayes the completely noncomparable m2 would beat the stinger on track as would a chevrolet or sportbike btw17 ford crew 27 67 coronet rt 14 towncountry limited 09 lr2 hse 44car history and counting'

In [91]:
temp = copy.deepcopy(comments_copy)

In [92]:
import nltk
from nltk import word_tokenize

In [93]:
temp['comments_appear'] = temp['comments_model_replace'].apply(word_tokenize).apply(set).apply(list)

# Remove stop words
stop_words = set(stopwords.words('english'))
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 
    
temp['final_comments'] =  temp['comments_appear'].apply(remove_stopwords)
#filtered_sentence = [w for w in temp['comments_appear'] if not w in stop_words]

In [94]:
count = []
for i in range(len(temp)):
    count+=temp['final_comments'][i]

In [95]:
from nltk import FreqDist
x = nltk.FreqDist(count)

In [96]:
models['brand'].drop_duplicates()

0              acura
9               audi
19               bmw
41             buick
49          cadillac
52               car
53         chevrolet
116         chrysler
125            dodge
137             ford
161            honda
207       hyndai kia
208          hyundai
248         infiniti
258              kia
268          lincoln
276            mazda
306         mercedes
313    mercedes benz
314     mercedesbenz
315          mercury
317       mitsubishi
319           nissan
338          pontiac
355          problem
356           saturn
357             seat
358            sedan
359           subaru
380           suzuki
409           toyata
410           toyota
508       volkswagen
522        volkwagen
523            volvo
Name: brand, dtype: object

In [82]:
word_freq[:100]

,word,freq
0,56,NaN
1,56,NaN
2,56,NaN
3,56,NaN
4,56,NaN
5,56,NaN
6,56,NaN
7,56,NaN
8,56,NaN
9,56,NaN
